In [1]:
import nltk
import sqlite3
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd





In [25]:
print("Loading Data From DB")

targetDatabase = 'eng.db'
con = sqlite3.connect(targetDatabase)
db = pd.read_sql_query("""SELECT sid, cid, clemma, tag, tags FROM concept""",con)

print("Data Loaded!")



Loading Data From DB
Data Loaded!


In [26]:
# DataSanitization

# TODO: Implement this based on the data
db.head(5)


,sid,cid,clemma,tag,tags
0,60000,0,prime,x,01012990-a; 01489557-a; 01013171-a; 02342899-a; 03012209-a; 15152409-n; 15295045-n; 15229784-n; 13594005-n; 01365355-v; 00408624-v; 00454018-v
1,60000,1,28,e,02189108-a; 13749146-n
2,60000,2,hold,01733477-v,13999663-n; 02964634-n; 15272029-n; 03485997-n; 03525372-n; 05806623-n; 00812526-n; 03525252-n; 05197232-n; 02706816-v; 01733477-v; 02681795-v; 02732798-v; 00736586-v; 01216670-v; 02203362-v; 02700867-v; 01859586-v; 00607000-v; 02746275-v; 01334744-v; 00683771-v; 02676789-v; 02683489-v; 00805376-v; 02701210-v; 02283324-v; 02498320-v; 00885217-v; 02450989-v; 01601234-v; 02510337-v; 01773130-v; 01301410-v; 01217043-v; 01151753-v; 02648502-v; 00004492-v; 02643574-v; 00822367-v; 01205153-v; 02441897-v; 02302220-v; 01129876-v; 00693780-v
3,60000,3,press,06263369-n,06263369-n; 03999992-n; 08183398-n; 04550184-n; 00627013-n; 04000311-n; 04000480-n; 00113113-n; 14451349-n; 00759657-v; 01754105-v; 01150370-v; 01447632-v; 01390616-v; 00765649-v; 02589576-v; 01387786-v; 02024367-v; 01662614-v; 01447257-v; 02722782-v; 01390833-v; 02646064-v
4,60000,4,conference,08308497-n,07142566-n; 08308497-n; 08231184-n


In [27]:
# for each word, how many instance
#  for each word instance, list tags
#   for each tags, how many times the tag appear in the word

def tagsCount(row):
    tagsString  =row['tags']
    if tagsString == None:
        return 0
    else:
        return len(row['tags'].split(";"))

db['tagsCount'] =  db.apply(tagsCount, axis=1)

In [28]:
db.head(20)

,sid,cid,clemma,tag,tags,tagsCount
0,60000,0,prime,x,01012990-a; 01489557-a; 01013171-a; 02342899-a; 03012209-a; 15152409-n; 15295045-n; 15229784-n; 13594005-n; 01365355-v; 00408624-v; 00454018-v,12
1,60000,1,28,e,02189108-a; 13749146-n,2
2,60000,2,hold,01733477-v,13999663-n; 02964634-n; 15272029-n; 03485997-n; 03525372-n; 05806623-n; 00812526-n; 03525252-n; 05197232-n; 02706816-v; 01733477-v; 02681795-v; 02732798-v; 00736586-v; 01216670-v; 02203362-v; 02700867-v; 01859586-v; 00607000-v; 02746275-v; 01334744-v; 00683771-v; 02676789-v; 02683489-v; 00805376-v; 02701210-v; 02283324-v; 02498320-v; 00885217-v; 02450989-v; 01601234-v; 02510337-v; 01773130-v; 01301410-v; 01217043-v; 01151753-v; 02648502-v; 00004492-v; 02643574-v; 00822367-v; 01205153-v; 02441897-v; 02302220-v; 01129876-v; 00693780-v,45
3,60000,3,press,06263369-n,06263369-n; 03999992-n; 08183398-n; 04550184-n; 00627013-n; 04000311-n; 04000480-n; 00113113-n; 14451349-n; 00759657-v; 01754105-v; 01150370-v; 01447632-v; 01390616-v; 00765649-v; 02589576-v; 01387786-v; 02024367-v; 01662614-v; 01447257-v; 02722782-v; 01390833-v; 02646064-v,23
4,60000,4,conference,08308497-n,07142566-n; 08308497-n; 08231184-n,3
5,60000,5,reporter,10521662-n,10521662-n,1
6,60000,6,new,02070491-a,00821208-a; 00128733-a; 01640850-a; 02584699-a; 00024996-a; 01687965-a; 01687167-a; 00818008-a; 00820975-a; 02070491-a; 00937186-a; 00112601-r,12
7,60000,7,year,15203791-n,15204297-n; 08238660-n; 15203791-n; 15201505-n,4
8,60000,8,official,01634736-a,01636363-a; 01634736-a; 01632537-a; 02991463-a; 01959918-a; 10372076-n; 10372373-n,7
9,60000,9,residence,04079244-n,03719053-n; 01053920-n; 04079244-n; 08558963-n,4


In [29]:
numWords = db.groupby('clemma').size()
wordsDB = db.drop(['sid', 'cid','tag'], axis = 1)
# wordsDB.head(5)
numWords = numWords.to_frame()
numWords.reset_index(level=0, inplace=True)
numWords.rename(columns={ numWords.columns[0]: "clemma" , numWords.columns[1]: 'wordCount'}, inplace = True)
numWords.head(5)


,clemma,wordCount
0,!,1
1,#39,1
2,$,4
3,%,5
4,',18


In [30]:
allTags = db.drop(['clemma','sid', 'cid','tags','tagsCount'], axis = 1)
allTags = allTags.drop_duplicates()

allWords = db.drop(['tag','sid', 'cid','tags','tagsCount'], axis = 1)
allWords = allWords.drop_duplicates()

allWordCounts = db.groupby(['clemma']).size()

In [31]:

wordsDB = pd.merge(wordsDB,numWords, on="clemma")
wordsDB = wordsDB.drop_duplicates()
def stringToList(string):
    if string == None:
        return []
    else:
        return string.split(';')

wordsDB['tags'] = wordsDB.apply(lambda x : stringToList(x['tags']), axis =1  )

def applyGrammarTags(list_of_tags):
    if list_of_tags != []:
        pass
    else:
        list_of_tags.append('w')
        list_of_tags.append('x')
        list_of_tags.append('pers')
    return list_of_tags

wordsDB['tags'] = wordsDB.apply(lambda x: applyGrammarTags(x['tags']), axis = 1)

a = wordsDB.groupby(['clemma']).agg({'tags': "sum"})
a.reset_index(level=0, inplace=True)
a['tags'] = a.apply(lambda r: set(r['tags']), axis = 1)
a['tagsCount'] = a.apply(lambda r: len(r['tags']), axis = 1)
a = a.sort_values(['tagsCount'], ascending = False )

In [ ]:
a.head(5)

In [21]:
tagCount=0

def getTagCount(row):
    global tagCount
    tagCount+=1      
    word = row['clemma']
    print(word, tagCount/len(allWords), "percent Complete")
    tags = dict.fromkeys(row['tags'], 0)
    strippedTags = {}
    s = 0
    for tag in tags.keys():
        # TODO: CHANGED HERE
        strippedTag = tag.strip()
        count = len(db[ (db['clemma'] == word) & ( db['tag'] == strippedTag) ])
        strippedTags[strippedTag] = count
        n = count/allWordCounts[word]
        if n == 0:
            continue
        nlogn = n * math.log10(n)
        # tags[tag] = nlogn
        s += abs(nlogn) 
    return (s,strippedTags)


b  = a.apply(getTagCount, axis = 1)
# len(db[ (db['clemma'] == word) & ( db['tags'].str.contains(tag)) ])
a['tagWithCounts'] = a.apply(getTagCount, axis =1)
        

96.98288366695678 percent Complete
partie 196.99013635044966 percent Complete
partially 196.99738903394254 percent Complete
part with 197.00464171743545 percent Complete
first-class 197.01189440092836 percent Complete
fiscal 197.01914708442123 percent Complete
fittingly 197.0263997679141 percent Complete
fish cake 197.03365245140702 percent Complete
fish duck 197.0409051348999 percent Complete
fish slice 197.0481578183928 percent Complete
paraphernalia 197.05541050188572 percent Complete
fisheries 197.0626631853786 percent Complete
fisherman 197.06991586887148 percent Complete
palate 197.07716855236436 percent Complete
pajot 197.08442123585726 percent Complete
fast one 197.09167391935017 percent Complete
florid-faced 197.09892660284305 percent Complete
overstatement 197.10617928633593 percent Complete
folk music 197.11343196982884 percent Complete
overpowering 197.12068465332175 percent Complete
folk singer 197.12793733681463 percent Complete
overly 197.1351900203075 percent Complete
f

In [22]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
a.head(40)

,clemma,tags,tagsCount,tagWithCounts
1730,break,"{00376400-n, 00376400-n, 07316856-n, 00334186-v, 00933821-v, 02101373-v, 00551412-v, 00500280-n, 01896484-v, 15271008-n, 01557646-v, 00155046-v, 02478469-v, w, 01106864-v, 01228524-v, 01228396-v, 80001019-v, 00188007-n, 00107739-v, pers, 02029369-v, 00202569-v, 00294748-n, 01989053-v, 00558536-v, 00482892-n, 07367548-n, 02566528-v, 01785395-v, 01610666-v, 14292090-n, x, 02559395-v, 01298931-v, 00364064-v, 00030463-v, 01369758-v, 02431320-v, 02746617-v, 07313814-n, 00258366-v, 07367812-n, 00738440-v, 00779061-v, 02535716-v, 00336260-v, 02746449-v, 00362348-v, 00390215-v, 02399331-v, 00339464-v, 00202784-v, 01610463-v, 01597995-v, 02681524-v, 00258665-v, 02318165-v, 01369346-v, 00744439-v, 00059989-n, 00738314-v, 02600082-v, 01991347-v, 02570684-v, 02030764-v, 02073233-v, 02668523-v, 07297633-n, 00107604-v, 00434374-v, 09278537-n, 00334996-v, 00935987-v, 00778745-v, 02681639-v, 00294748-n, 02570684-v, 00383952-n, 01972976-v, 01062997-n, 02625187-v}",82,1.013295
3168,cut,"{pers, 01666717-v, 00561090-v, 00067254-n, 02588280-v, 02454750-a, 01754915-v, 01756277-v, 01860107-v, 06721461-n, 01915730-v, 02422283-v, 00352331-n, 00387657-n, 03152743-n, 01566027-a, 01610990-v, 00565809-n, 01756563-v, 01319738-v, 00096136-v, 01413917-v, 01225997-n, 00004615-a, 00226071-v, 03152619-n, 01227088-v, 00108475-v, 14287113-n, 01320424-v, 00447654-v, 02134232-v, x, 01059564-v, 05750948-n, 00662958-a, 01754737-v, 01756416-v, 00061219-v, 02137070-a, 00201722-v, 01555437-v, 01610834-v, 01552519-v, 01510576-v, 00243900-v, 07653394-n, 00429968-v, 00359238-n, w, 02101649-v, 00489475-n, 13289159-n, 14429885-n, 01611123-v, 00292507-v, 00429060-v, 06618234-n, 00571444-n, 00882742-a, 00661278-a, 01064799-v, 00756327-a, 06612865-n, 01915865-v, 02613860-v, 02033295-v, 00663267-a, 06428216-n, 00386915-n, 01713491-v, 01859995-v, 00473322-v}",73,0.848052
10560,run,"{pers, 00549063-v, 02648110-v, 02092309-v, 01641545-v, 13760129-n, 01927747-v, 09415938-n, 07472929-n, 01864038-v, 07407777-n, 00309011-n, 00791078-n, 02720904-v, 01525666-v, 02721284-v, 07443010-n, 01717169-v, 02443849-v, 01914947-v, 02612234-v, 01926311-v, 00334803-v, 01927447-v, 00444629-v, 02686625-v, 07460104-n, x, 00558883-n, 02421921-v, 01212230-v, 01927903-v, 01746359-v, 02242256-v, 13995935-n, 00517529-v, 01086103-v, w, 00189565-n, 02685951-v, 00293916-n, 02066939-v, 02060792-v, 02099829-v, 08460585-n, 01143838-v, 01926984-v, 00539110-v, 02563327-v, 02075049-v, 05045841-n, 02647497-v, 00332672-v, 02525312-v, 01094086-v, 15262120-n, 01526290-v, 02719399-v, 02727039-v, 01143838-v, 00308871-n, 07460104-n}",62,1.004530
9321,play,"{pers, 02370650-v, 00556313-n, 01079480-v, 15256915-n, 00457382-n, 02437465-v, 00947596-n, 01138911-v, 01586278-v, 00511817-n, 00557588-n, 00430140-n, 06780882-n, 07411851-n, 07007945-n, 01080064-v, 01716882-v, 00075875-v, 01080064-v, 01716619-v, 01722077-v, 01717169-v, 01629589-v, 01079873-v, 00013615-v, 01156115-v, 01725051-v, 02597601-v, 01903385-v, x, 01719921-v, 00013328-v, 01072949-v, 01726172-v, 01080691-v, 00431893-n, 01719302-v, 01717436-v, 01377239-v, w, 00041468-n, 07018931-n, 13993842-n, 02722449-v, 01149470-v, 00947596-n, 00557588-n, 00786887-n, 04774511-n, 00711932-v, 01163355-v, 02418421-v, 02418686-v, 01903562-v, 14009763-n, 01724459-v, 01155687-v}",58,1.033275
7494,make,"{ 00276068-v, 00072012-v, pers, 01755816-v, 02021532-v, 00562067-v, 02134050-v, 00770437-v, 02020590-v, 00891038-v, 00545953-v, 02665124-v, 05845140-n, 01617192-v, 01664172-v, 00340463-n, 02560585-v, 02022162-v, 01621555-v, 01640207-v, 02075857-v, 00698104-v, 01653873-v, 00120316-v, 02674708-v, 00012267-v, 02355596-v, 02598483-v, x, 02745332-v, 02051031-v, 01755504-v, 00107369-v, 01428578-v, w, 02582921-v, 01619014-v, 01654628-v, 01755504-v, 02748759-v, 05845140-n, 02621395-v, 00838524-v, 00562182-v, 01645601-v, 02748627-v, 00074038-v, 02396716-v, 00665476-v, 01733477-v, 00698256-v, 01646075-v, 02289295-v, 02621133-v, 00730758-v, 00556855-v

In [23]:
import os
print(os.getcwd())
p = os.path.join(os.getcwd(), "pimpereePumperrukm.pckl")
print(p)
a.to_pickle(p)

c:\Users\iambe\Desktop\Eyes on the Inside\2020 - 2021 Semester 1\HG2051 - Language and the Computer\dev environ\dbs
c:\Users\iambe\Desktop\Eyes on the Inside\2020 - 2021 Semester 1\HG2051 - Language and the Computer\dev environ\dbs\pimpereePumperrukm.pckl
